# 1.1.3.3: Eigenvectores y Eigenvalores

## Objetivos de Aprendizaje

Al completar este notebook, serás capaz de:

- **Definir** un eigenvector y un eigenvalor con la ecuación fundamental $A\vec{v} = \lambda\vec{v}$.
- **Interpretar** geométricamente a los eigenvectores como los **ejes de inercia o ejes principales de una transformación**.
- **Calcular** los eigenvalores y eigenvectores de una matriz usando NumPy.
- **Comprender** el concepto de **eigendescomposición** de una matriz.
- **Aplicar** la eigendescomposición a una matriz de covarianza para encontrar los **Componentes Principales (PCA)** de un dataset, la culminación de nuestro viaje por el álgebra lineal.

In [ ]:
# --- Celda de Configuración (Oculta) ---
%display latex
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def plot_eigen_transformation(matrix, vectors, ax=None, title='Transformación de Vectores'):
    standalone = ax is None
    if standalone:
        fig, ax = plt.subplots(figsize=(8, 8))
    
    colors = ['#0072B2', '#E69F00', '#D55E00', '#CC79A7']
    
    for i, v in enumerate(vectors):
        color = colors[i % len(colors)]
        transformed_v = matrix @ v
        # Vector original
        ax.quiver(0, 0, v[0], v[1], angles='xy', scale_units='xy', scale=1, 
                  color=color, label=f'v{i+1}')
        # Vector transformado
        ax.quiver(0, 0, transformed_v[0], transformed_v[1], angles='xy', scale_units='xy', scale=1, 
                  color=color, linestyle='--', alpha=0.7, label=f'A·v{i+1}')
    
    limit = np.max(np.abs(np.hstack([vectors, matrix@np.array(vectors).T]))) * 1.2
    ax.set_xlim(-limit, limit); ax.set_ylim(-limit, limit)
    ax.set_aspect('equal'); ax.grid(True, linestyle='--')
    ax.axhline(0, c='black', lw=0.5); ax.axvline(0, c='black', lw=0.5)
    ax.legend(); ax.set_title(title)
    
    if standalone:
        plt.show()

--- 
## ⚙️ El Arsenal de Datasets: Nuestra Fuente de Ejercicios

Para este tema culminante, usaremos datasets que tengan una estructura de covarianza interesante. El objetivo es descubrir los "ejes" ocultos de estos datos. Nuestro generador de matrices especiales también será crucial para crear transformaciones con eigenvectores predecibles.

In [ ]:
# === CONFIGURACIÓN DE DATASETS ===
from src.data_generation.create_student_performance import create_student_performance_data
from src.data_generation.create_special_matrices import create_special_matrices
from src.data_generation.create_edge_cases import create_edge_cases

# Configuración centralizada de aleatoriedad para REPRODUCIBILIDAD
rng = np.random.default_rng(seed=42)

# === Generación de Datasets y Matrices para este Notebook ===

# 💡 CONTEXTO PEDAGÓGICO: Hilo Conductor (Aplicación a PCA)
# Este es el gran final. Aplicaremos la eigendescomposición a la matriz de covarianza
# de nuestros estudiantes para encontrar los Componentes Principales (PCA) de los datos.
datos_estudiantes = create_student_performance_data(rng, simplified=True, n_samples=200)

# 💡 CONTEXTO PEDAGÓGICO: Matrices con Ejes Claros
# Una matriz simétrica es ideal para ilustrar eigenvectores, ya que sus eigenvectores
# son siempre ortogonales y representan los ejes de una elipse de transformación.
matriz_simetrica = create_special_matrices(rng, matrix_type='symmetric', size=(2, 2))

# 💡 CONTEXTO PEDAGÓGICO: Datos Correlacionados para PCA
# Generaremos datos con una fuerte correlación para que PCA pueda encontrar de forma
# muy evidente el eje principal de la varianza.
datos_correlacionados = create_edge_cases(rng, case_type='multicollinear', n_samples=200)[['x1', 'x2']]

print("Datasets y matrices generados y listos para usar.")

## 1. La Idea Intuitiva: "Los Ejes de una Transformación"

Imagina una transformación lineal como un estiramiento o compresión del espacio. La mayoría de los vectores cambiarán tanto su longitud como su dirección. Sin embargo, existen unos vectores "especiales" que son privilegiados: su dirección no cambia, solo son escalados (estirados o encogidos).

- **Eigenvectores ($\vec{v}$):** Los vectores cuya dirección no cambia bajo la transformación $A$. Son los **ejes de acción** de la matriz.
- **Eigenvalores ($\lambda$):** El factor de escala (un escalar) correspondiente a cada eigenvector. Nos dice *cuánto* se estira o encoge el eigenvector.

Esta relación única se captura en la ecuación más famosa del álgebra lineal:
$$ A\vec{v} = \lambda\vec{v} $$

### El Proceso de Cálculo

Para encontrar estos valores, reorganizamos la ecuación a $(A - \lambda I)\vec{v} = \vec{0}$. Para que exista una solución no trivial para $\vec{v}$ (es decir, $\vec{v} \neq \vec{0}$), la matriz $(A - \lambda I)$ debe ser singular. Esto significa que su determinante debe ser cero, lo que nos lleva a la **Ecuación Característica**:
$$ \det(A - \lambda I) = 0 $$
Resolviendo esta ecuación para $\lambda$ obtenemos los eigenvalores. Luego, para cada $\lambda$, resolvemos el sistema de ecuaciones para encontrar el $\vec{v}$ correspondiente (el eigenvector), que formará el kernel (espacio nulo) de $(A - \lambda I)$.

### Ejemplo Demostrativo 1: Cálculo y Verificación Visual

In [ ]:
# 1. DATOS: Usamos nuestra matriz simétrica generada.
A = matriz_simetrica

# 2. APLICACIÓN: Calculamos los eigenvalores y eigenvectores con NumPy.
eigenvalores, eigenvectores = np.linalg.eig(A)
lambda1, lambda2 = eigenvalores
v1, v2 = eigenvectores[:, 0], eigenvectores[:, 1]

# 3. INTERPRETACIÓN
print(f"Matriz de Transformación A:\n{np.round(A, 2)}")
print(f"\nEigenvalor 1 (λ1): {lambda1:.2f} con Eigenvector v1: {np.round(v1, 2)}")
print(f"Eigenvalor 2 (λ2): {lambda2:.2f} con Eigenvector v2: {np.round(v2, 2)}")

# Verificamos la ecuación A·v = λ·v para el primer par
Av1 = A @ v1
lambda1_v1 = lambda1 * v1
print(f"\nVerificación para v1: A·v1 = {np.round(Av1, 2)}, λ1·v1 = {np.round(lambda1_v1, 2)}")
print(f"¿Son iguales? {np.allclose(Av1, lambda1_v1)}")

# 4. VISUALIZACIÓN
# También incluimos un vector que NO es un eigenvector para ver la diferencia.
v_no_eigen = np.array([1, 0])
plot_eigen_transformation(A, [v1, v2, v_no_eigen], title='v1 y v2 (sólidos) son eigenvectores, v3 no lo es.')

## 2. Aplicación Estrella: Análisis de Componentes Principales (PCA)

Llegamos al punto donde todos los conceptos se unen. El PCA es una técnica de reducción de dimensionalidad que busca encontrar las "direcciones de máxima varianza" en un conjunto de datos. Estas direcciones son, precisamente, los **eigenvectores de la matriz de covarianza** del dataset.

- **Eigenvectores (Componentes Principales):** Nos dan los nuevos ejes (ortogonales) sobre los cuales proyectar los datos.
- **Eigenvalores:** Nos dicen cuánta varianza de los datos es "explicada" por cada uno de esos ejes. Un eigenvalor grande corresponde a un componente principal importante.

### Ejemplo Demostrativo 2: Encontrando los Ejes Principales de Nuestros Datos

In [ ]:
# 1. PREPARAR LOS DATOS: PCA requiere que los datos estén centrados en el origen.
datos = datos_estudiantes[['horas_estudio', 'calificacion_examen']].values
datos_centrados = datos - np.mean(datos, axis=0)

# 2. CALCULAR LA MATRIZ DE COVARIANZA
# Esta matriz 2x2 describe la varianza y covarianza de nuestras dos features.
cov_matrix = np.cov(datos_centrados, rowvar=False)

# 3. EIGENDESCOMPOSICIÓN DE LA MATRIZ DE COVARIANZA
eigenvalores, eigenvectores = np.linalg.eig(cov_matrix)

# Ordenamos de mayor a menor eigenvalor
idx = eigenvalores.argsort()[::-1]
eigenvalores = eigenvalores[idx]
eigenvectores = eigenvectores[:, idx]

# 4. INTERPRETACIÓN
print("Matriz de Covarianza:")
print(np.round(cov_matrix, 2))
print(f"\nEigenvalores (Varianza explicada por cada eje): {np.round(eigenvalores, 2)}")
print(f"Eigenvectores (Componentes Principales):\n{np.round(eigenvectores, 2)}")

varianza_explicada = eigenvalores / np.sum(eigenvalores)
print(f"\nEl primer componente principal explica el {varianza_explicada[0]:.1%} de la varianza total.")

# 5. VISUALIZACIÓN
fig, ax = plt.subplots(figsize=(9, 9))
ax.scatter(datos_centrados[:, 0], datos_centrados[:, 1], alpha=0.5, label='Datos de Estudiantes (Centrados)')

# Dibujamos los eigenvectores escalados por sus eigenvalores
for i in range(eigenvectores.shape[1]):
    vec = eigenvectores[:, i]
    # Escalar para visualización: 3 * sqrt(eigenvalor) para que se vea como 3 desviaciones estándar
    scaled_vec = vec * 3 * np.sqrt(eigenvalores[i]) 
    ax.quiver(0, 0, scaled_vec[0], scaled_vec[1], angles='xy', scale_units='xy', scale=1, 
              color=['#D55E00', '#009E73'][i], width=0.01, label=f'Componente Principal {i+1}')

ax.set_aspect('equal'); ax.grid(True, linestyle='--'); ax.legend()
ax.set_title("PCA: Eigenvectores de la Matriz de Covarianza")
plt.show()

---
## 4. Ejercicios Guiados con Scaffolding (8+)
Rellena las partes marcadas con `# COMPLETAR` para afianzar tu comprensión.

### === EJERCICIO GUIADO 1: Verificar un Eigenvector ===

In [ ]:
# DATOS
A = np.array([[5, -1], [3, 1]])
v_candidato = np.array([1, 1])
lambda_candidato = 4

# TODO 1: Calcula el lado izquierdo de la ecuación: A @ v
lado_izquierdo = # COMPLETAR

# TODO 2: Calcula el lado derecho de la ecuación: λ * v
lado_derecho = # COMPLETAR

# TODO 3: Compara si ambos lados son (aproximadamente) iguales.
es_eigenvector = # COMPLETAR

# VERIFICACIÓN
assert es_eigenvector, "El vector candidato debería ser un eigenvector con el eigenvalor dado."
print(f"✅ ¡Correcto! A·v = {lado_izquierdo} y λ·v = {lado_derecho}, por lo que v es un eigenvector de A.")

### === EJERCICIO GUIADO 2: Calcular Eigenvalores y Eigenvectores ===

In [ ]:
# DATOS
A = np.array([[2, 7], [7, 2]])

# TODO 1: Usa np.linalg.eig() para obtener los eigenvalores y eigenvectores de A.
eigenvalores, eigenvectores = # COMPLETAR

# VERIFICACIÓN
assert eigenvalores.shape == (2,), "Debería haber 2 eigenvalores."
assert eigenvectores.shape == (2, 2), "Debería haber 2 eigenvectores de 2 dimensiones."
# Verificamos la ecuación para el primer par
assert np.allclose(A @ eigenvectores[:, 0], eigenvalores[0] * eigenvectores[:, 0])
print("✅ ¡Cálculo correcto!")
print(f"Eigenvalores: {eigenvalores}")
print(f"Eigenvectores (como columnas):\n{eigenvectores}")

### === EJERCICIO GUIADO 3: Eigenvalores de una Matriz de Proyección ===

In [ ]:
# DATOS: Una matriz que proyecta vectores sobre el eje X.
P = np.array([[1, 0], [0, 0]])

# TODO 1: Piensa geométricamente. ¿Qué vectores no cambian de dirección al ser proyectados sobre el eje X?
# ¿Y qué vectores son 'aplastados' a cero? Esto te dará los eigenvectores y eigenvalores.
eigenvalor1_esperado = # COMPLETAR (Para vectores ya en el eje X)
eigenvalor2_esperado = # COMPLETAR (Para vectores en el eje Y)

# TODO 2: Calcula los eigenvalores numéricamente para confirmar.
eigenvalores, _ = # COMPLETAR

# VERIFICACIÓN
assert eigenvalor1_esperado == 1
assert eigenvalor2_esperado == 0
assert sorted(eigenvalores.tolist()) == [0, 1]
print("✅ ¡Análisis correcto!")
print("Un vector en el eje X se queda igual (λ=1), un vector en el eje Y es aplastado a cero (λ=0).")

### === EJERCICIO GUIADO 4: Eigendescomposición (A = P·D·P⁻¹) ===

In [ ]:
# DATOS: La misma matriz simétrica A del ejercicio guiado 2.
A = np.array([[2, 7], [7, 2]])

# TODO 1: Obtén los eigenvalores (D) y eigenvectores (P).
lambdas, P = # COMPLETAR

# TODO 2: Construye la matriz diagonal D.
# PISTA: Usa np.diag().
D = # COMPLETAR

# TODO 3: Calcula la inversa de P.
# PISTA: Usa np.linalg.inv().
P_inv = # COMPLETAR

# TODO 4: Reconstruye la matriz original multiplicando P @ D @ P_inv.
A_reconstruida = # COMPLETAR

# VERIFICACIÓN
assert np.allclose(A, A_reconstruida)
print("✅ ¡Eigendescomposición verificada!")
print(f"La matriz original A puede ser reconstruida a partir de sus eigenvectores y eigenvalores.")

--- 
# 5. Banco de Ejercicios Prácticos (30+)
Ahora te toca a ti. Resuelve estos ejercicios para consolidar tu conocimiento.

### Parte A: Cálculo y Verificación

**A1 (🟢 Fácil):** Encuentra los eigenvalores y eigenvectores de $A = \begin{pmatrix} 2 & 1 \\ 1 & 2 \end{pmatrix}$.

**A2 (🟢 Fácil):** Encuentra los eigenvalores y eigenvectores de $B = \begin{pmatrix} 5 & 0 \\ 0 & -3 \end{pmatrix}$. ¿Qué observas?

**A3 (🟡 Medio):** Verifica que $\vec{v}=[1,1]$ es un eigenvector de $M = \begin{pmatrix} 4 & 1 \\ 2 & 3 \end{pmatrix}$. ¿Cuál es su eigenvalor correspondiente?

**A4 (🟡 Medio):** Genera una matriz simétrica 3x3. Calcula sus eigenvalores y eigenvectores.

**A5 (🔴 Reto):** La traza de una matriz (la suma de su diagonal) es igual a la suma de sus eigenvalores. El determinante es igual al producto de sus eigenvalores. Genera una matriz 3x3 aleatoria y verifica estas dos propiedades.

### Parte B: Interpretación Geométrica

**B1 (🟢 Fácil):** Una matriz de reflexión sobre el eje y es $F = \begin{pmatrix} -1 & 0 \\ 0 & 1 \end{pmatrix}$. ¿Cuáles son sus eigenvectores y eigenvalores? Piénsalo geométricamente antes de calcular.

**B2 (🟢 Fácil):** ¿Cuáles son los eigenvalores de la matriz identidad 2x2? ¿Y sus eigenvectores?

**B3 (🟡 Medio):** Una matriz de cizalla (shear) es $S = \begin{pmatrix} 1 & 1 \\ 0 & 1 \end{pmatrix}$. Calcula sus eigenvectores y eigenvalores. ¿Por qué solo hay un eje de eigenvectores?

**B4 (🔴 Reto):** Una matriz de rotación de 45 grados en 2D no tiene eigenvectores reales (ningún vector real mantiene su dirección). Calcula sus eigenvalores y observa que son números complejos.

### Parte C: Aplicación a PCA

**C1 (🟡 Medio):** Toma el dataset `datos_correlacionados` que generamos. Céntralo, calcula su matriz de covarianza y encuentra sus eigenvectores y eigenvalores.

**C2 (🟡 Medio):** Para el ejercicio C1, ¿qué porcentaje de la varianza total es explicada por el primer componente principal (el eigenvector con el mayor eigenvalor)?

**C3 (🔴 Reto):** En el Ejemplo 2 (PCA), el primer componente principal es el primer eigenvector (`pc1 = eigenvectores[:, 0]`). Proyecta todos los `datos_centrados` sobre la línea definida por este vector. La fórmula para las proyecciones es `proyecciones = datos_centrados @ pc1`. El resultado será un array 1D. Grafica un histograma de estas proyecciones.

---

## ✅ Mini-Quiz de Autoevaluación

*Responde estas preguntas para verificar tu comprensión.*

1. ¿Qué ecuación se debe resolver para encontrar los eigenvalores de una matriz A?
2. En el contexto de PCA, ¿qué representan los eigenvectores de la matriz de covarianza?
3. Si un eigenvalor es 0, ¿qué puedes concluir sobre el determinante de la matriz y su invertibilidad?
4. Verdadero o Falso: Una matriz 3x3 siempre tiene 3 eigenvectores linealmente independientes.

## 🚀 Próximos Pasos (Cierre del Área 1.1)

**¡Felicidades, has completado los fundamentos del Álgebra Lineal!** Has construido un entendimiento sólido desde la definición de un vector hasta la aplicación de eigenvectores en una de las técnicas más importantes de la ciencia de datos: PCA. Ahora tienes la base conceptual y computacional para entender cómo se manipulan los datos en alta dimensión.

- En el siguiente gran bloque, **Área 1.2: Cálculo**, cambiaremos de marcha para explorar la matemática del cambio continuo. Descubriremos cómo las derivadas nos permiten encontrar la "pendiente" en espacios de alta dimensión, una idea fundamental para entrenar y optimizar casi todos los modelos de Machine Learning.